In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
cur_path = "/content/drive/MyDrive/SNA/final_project/"
os.chdir(cur_path)
!pwd

Mounted at /content/drive
/content/drive/MyDrive/SNA/final_project


In [2]:
%%capture
# Environment Setup (note that capture silences the console output)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git


In [3]:
!pip install scrapbook
!pip install recommenders

     |████████████████████████████████| 130 kB 12.4 MB/s 
     |████████████████████████████████| 428 kB 35.4 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=5.1.0; python_version >= "3.0", but you have tornado 6.1 which is incompatible.


     |████████████████████████████████| 335 kB 5.2 MB/s 
     |████████████████████████████████| 310 kB 12.4 MB/s 
     |████████████████████████████████| 1.5 MB 11.8 MB/s 
     |████████████████████████████████| 197 kB 26.0 MB/s 
     |████████████████████████████████| 11.8 MB 15.1 MB/s 
     |████████████████████████████████| 4.0 MB 43.1 MB/s 
     |████████████████████████████████| 636 kB 46.4 MB/s 
     |████████████████████████████████| 61 kB 6.1 MB/s 
     |████████████████████████████████| 12.4 MB 32.6 MB/s 
     |████████████████████████████████| 749 kB 41.2 MB/s 
     |████████████████████████████████| 10.9 MB 39.6 MB/s 
     |████████████████████████████████| 383 kB 37.2 MB/s 
     |████████████████████████████████| 880 kB 44.5 MB/s 
     |████████████████████████████████| 6.6 MB 43.8 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705388 sha256=d8f2cc300267eb8c69014507acdc0d99d9

In [4]:
# set the environment path to find Recommenders
from tempfile import TemporaryDirectory
import sys
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import scrapbook as sb
import torch, fastai
from fastai.collab import collab_learner, CollabDataBunch, load_learner

from recommenders.utils.constants import ( 
    DEFAULT_PREDICTION_COL as PREDICTION
) 
from recommenders.utils.timer import Timer
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.models.fastai.fastai_utils import cartesian_product, score
from recommenders.evaluation.python_evaluation import map_at_k, precision_at_k, recall_at_k
from recommenders.evaluation.python_evaluation import rmse, mae

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Fast AI version: {}".format(fastai.__version__))
print("Torch version: {}".format(torch.__version__))
print("Cuda Available: {}".format(torch.cuda.is_available()))
print("CuDNN Enabled: {}".format(torch.backends.cudnn.enabled))


System version: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
Pandas version: 1.3.5
Fast AI version: 1.0.61
Torch version: 1.11.0+cu113
Cuda Available: True
CuDNN Enabled: True


# Running on GPU

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Loading in Data
## Loading in 20,000 rows, entire data set is too large.

In [6]:
df = pd.read_csv("transactions_train.csv.zip", nrows=20000)
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


### Defning number of items, number of factors, and epochs

In [7]:
# top k items to recommend
TOP_K = 12

# Model parameters
factors = 60
epochs = 10

### Sorting the customers by the number of purchases they made so we have top customers in dataset
### Then taking the 1000 best customers

In [8]:
df['customer_purchases']=df.groupby('customer_id')['article_id'].count().reset_index(name='count').sort_values(['count'], ascending=False).iloc[:1000,1]

In [9]:
sample=df.loc[pd.notna(df.customer_purchases)].reset_index()

In [10]:
# make sure the IDs are loaded as strings to better prevent confusion with embedding ids
sample['customer_id'] = sample['customer_id'].astype('str')
sample['article_id'] = sample['article_id'].astype('str')
sample.drop(columns=['index'], inplace=True)
sample.head()

,t_dat,customer_id,article_id,price,sales_channel_id,customer_purchases
0,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,30.0
1,2018-09-20,0008968c0d451dbc5a9968da03196fe20051965edde741...,531310002,0.022525,2,11.0
2,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,685687002,0.016932,2,6.0
3,2018-09-20,001fd23db1109a94bba1319bb73df0b479059027c182da...,562252035,0.021593,1,6.0
4,2018-09-20,0021da829b898f82269fc51feded4eac2129058ee95bd7...,629760002,0.015237,1,6.0


# Splitting the data

In [11]:
train, test = train_test_split(sample.values, test_size=0.3, random_state=1) # 70/30 Split
train_df = pd.DataFrame(train, columns=df.columns)
test_df = pd.DataFrame(test, columns=df.columns)
print("Train Size  : ", len(train_df))
print("Test Size : ", len (test_df))

Train Size  :  700
Test Size :  300


# Prepping the data

### Creating negative interactions by filling in what customers did not buy.

In [12]:
def prep_data(df):

  df = df.drop_duplicates()
  df1 = df[['customer_id', 'article_id']].copy()
  df1['Feedback'] = 1
  df1 = df1.drop_duplicates()
  customers = df['customer_id'].unique() # take unique customers
  articles = df['article_id'].unique() # take unique items
  interaction_lst = []
  for customer in customers: # create a list that creates every possible combination of the two, with a feedback of 0
      for article in articles:
          interaction_lst.append([customer, article, 0])

  data_all = pd.DataFrame(data=interaction_lst, columns=["customer_id", "article_id", "FeedbackAll"]) # put into dataframe
  data_final = pd.merge(data_all, df1, on=['customer_id', 'article_id'], how='outer').fillna(0).drop('FeedbackAll', axis=1) # join together, this keeps the values of feedback 1 and fills in all items with 0 the customer did not interact with

  return data_final


In [13]:
train_df=prep_data(train_df)

In [14]:
test_df=prep_data(test_df)

In [15]:
test_df = test_df[test_df.customer_id.isin(train_df.customer_id)] # cold start problem, have to make sure they are in the training and test


In [16]:
train_df

,customer_id,article_id,Feedback
0,1883113cd5440f290d292cf4af84e51c63032bb9613463...,452618001,1.0
1,1883113cd5440f290d292cf4af84e51c63032bb9613463...,662868003,0.0
2,1883113cd5440f290d292cf4af84e51c63032bb9613463...,632172003,0.0
3,1883113cd5440f290d292cf4af84e51c63032bb9613463...,657476002,0.0
4,1883113cd5440f290d292cf4af84e51c63032bb9613463...,651724003,0.0
...,...,...,...
327707,022b4a549932e91686ed9cf3ffbacc2422343f5bc05012...,608007006,0.0
327708,022b4a549932e91686ed9cf3ffbacc2422343f5bc05012...,555470006,1.0
327709,022b4a549932e91686ed9cf3ffbacc2422343f5bc05012...,630116005,0.0
327710,022b4a549932e91686ed9cf3ffbacc2422343f5bc05012...,622877004,0.0


In [17]:
with Timer() as preprocess_time:
    data = CollabDataBunch.from_df(train_df, 
                                   user_name='customer_id', 
                                   item_name='article_id', 
                                   rating_name='Feedback', 
                                   valid_pct=0) # we did the splitting ourself so we use 0
                                   # This doesn't really do much besides do some of the heavy lifting for the formatting

In [18]:
data.show_batch()

customer_id,article_id,target
08efb51bd6a41dd3135a3c548e8e1411a36b688a4a934ba992bfa8c6767e2e44,620870004,0.0
16edb053b32a901e5efa26f73df904be92045f3e7019e517f866140c0633ba20,467302100,0.0
13fe5dd07e6e9483465ed2128fcbcbeec166214715d3848ae17c7d630891d30a,635418002,0.0
198283e9983c9257747407551439eee3642c77cf3c52ac7de225da9825e61f8b,111593001,0.0
1eacddce0e5ecd3a945b63f7ded47d2af4471ed7a1d75e0e276b760701c7e60a,554541016,0.0


### Creating the embeddings

In [19]:
learn = collab_learner(data, n_factors=factors, y_range=[-.5,1.5], wd=1e-2) # creating the embedding dot bais, using the fed number of factors
                                                                          # We set a y_range of -.5 and 1.5 because it bases the output on integers which in our case
                                                                          # will be 0 and 1, negative and positive
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(533, 60)
  (i_weight): Embedding(617, 60)
  (u_bias): Embedding(533, 1)
  (i_bias): Embedding(617, 1)
)

# Training

In [20]:
with Timer() as train_time:
    learn.fit_one_cycle(epochs, max_lr=.001) # This runs for set number of epochs

print("Took {} seconds for training.".format(train_time)) # Doubling the sample roughly quadruples the training time 

epoch,train_loss,valid_loss,time
0,0.002884,#na#,01:01
1,0.000891,#na#,01:01
2,0.003223,#na#,01:01
3,0.002079,#na#,01:02
4,0.002054,#na#,01:03
5,0.001856,#na#,01:02
6,0.002605,#na#,01:02
7,0.001935,#na#,01:01
8,0.002281,#na#,01:01
9,0.002658,#na#,01:01


Took 620.2829 seconds for training.


In [21]:
tmp = TemporaryDirectory()
model_path = os.path.join(tmp.name, "model.pkl")
learn.export(model_path)
learner = load_learner(tmp.name, "model.pkl")

# Validation

In [22]:
all_users, all_items = learner.data.train_ds.x.classes.values()
all_items = all_items[1:]
all_users = all_users[1:]
unique_users_test = test_df['customer_id'].unique()
users_in_test = np.intersect1d(unique_users_test, all_users) # ensuring users are in both train and test

In [23]:
users_item_pairs = cartesian_product(np.array(users_in_test),np.array(all_items)) # set of all ordered pairs
print(users_item_pairs)
users_item_pairs = pd.DataFrame(users_item_pairs, columns=['customer_id','article_id'])

[['00401a367c5ac085cb9d4b77c56f3edcabf25153615db982fcc7991d182e10a9' '108775051']
 ['00401a367c5ac085cb9d4b77c56f3edcabf25153615db982fcc7991d182e10a9' '111593001']
 ['00401a367c5ac085cb9d4b77c56f3edcabf25153615db982fcc7991d182e10a9' '156231001']
 ['00401a367c5ac085cb9d4b77c56f3edcabf25153615db982fcc7991d182e10a9' '179123001']
 ...
 ['1f4fac36cbae42bf927c001187c9cd46ddb757f33f31cc1d39bdd00c04a93231' '716519001']
 ['1f4fac36cbae42bf927c001187c9cd46ddb757f33f31cc1d39bdd00c04a93231' '717341001']
 ['1f4fac36cbae42bf927c001187c9cd46ddb757f33f31cc1d39bdd00c04a93231' '719031001']
 ['1f4fac36cbae42bf927c001187c9cd46ddb757f33f31cc1d39bdd00c04a93231' '729931001']]


In [24]:
train_erased = pd.merge(users_item_pairs, test_df.astype(str), on=['customer_id','article_id'], how='left')  # removing users in the training
print(train_erased)
train_erased = train_erased[pd.notna(train_erased['Feedback'])][['customer_id','article_id']]
print(train_erased)

                                             customer_id article_id Feedback
0      00401a367c5ac085cb9d4b77c56f3edcabf25153615db9...  108775051      NaN
1      00401a367c5ac085cb9d4b77c56f3edcabf25153615db9...  111593001      NaN
2      00401a367c5ac085cb9d4b77c56f3edcabf25153615db9...  156231001      NaN
3      00401a367c5ac085cb9d4b77c56f3edcabf25153615db9...  179123001      NaN
4      00401a367c5ac085cb9d4b77c56f3edcabf25153615db9...  186262006      NaN
...                                                  ...        ...      ...
65291  1f4fac36cbae42bf927c001187c9cd46ddb757f33f31cc...  714423001      NaN
65292  1f4fac36cbae42bf927c001187c9cd46ddb757f33f31cc...  716519001      NaN
65293  1f4fac36cbae42bf927c001187c9cd46ddb757f33f31cc...  717341001      0.0
65294  1f4fac36cbae42bf927c001187c9cd46ddb757f33f31cc...  719031001      NaN
65295  1f4fac36cbae42bf927c001187c9cd46ddb757f33f31cc...  729931001      NaN

[65296 rows x 3 columns]
                                             custo

# Validation Results

Metrics described in paper



In [25]:
with Timer() as test_time:
    top_k_scores = score(learner, 
                         test_df=train_erased,
                         user_col='customer_id', 
                         item_col='article_id', 
                         prediction_col=PREDICTION)

print("Took {} seconds for {} predictions.".format(test_time, len(train_erased)))

Took 0.0228 seconds for 6148 predictions.


### Built in Metric Functions

In [26]:
map = map_at_k(test_df,
                    top_k_scores,
                    col_user='customer_id',
                    col_item='article_id', 
                    col_rating='Feedback',
                    col_prediction=PREDICTION, 
                    relevancy_method="top_k",
                    k=TOP_K)

precision = precision_at_k(test_df, top_k_scores,
                                col_user='customer_id',
                                col_item='article_id',
                                col_rating='Feedback',
                                col_prediction=PREDICTION,
                                relevancy_method="top_k",
                                k=TOP_K)
recall = recall_at_k(test_df,
                          top_k_scores,
                          col_user='customer_id',
                          col_item='article_id', 
                          col_rating='Feedback',
                          col_prediction=PREDICTION,
                          relevancy_method="top_k",
                          k=TOP_K)

In [27]:
print("Model:\t" + learn.__class__.__name__,
      "Top K:", TOP_K,
      "Mean Average Percision:", map,
      "Precision@K: ", precision,
      "Recall@K: ", recall, sep='\n')

Model:	CollabLearner
Top K:
12
Mean Average Percision:
0.04240282685512369
Precision@K: 
1.0
Recall@K: 
0.04240282685512369


In [28]:
scores = score(learner, 
               test_df=top_k_scores, 
               user_col='customer_id', 
               item_col='article_id', 
               prediction_col=PREDICTION)

In [29]:
rmse = rmse(test_df,
                 scores,
                 col_user='customer_id',
                 col_item='article_id',
                 col_rating='Feedback',
                 col_prediction=PREDICTION)
mae = mae(test_df,
               scores,
               col_user='customer_id',
               col_item='article_id',
               col_rating='Feedback',
               col_prediction=PREDICTION)

print("Model:\t" + learn.__class__.__name__,
      "Root Mean Square Error:", rmse,
      "Mean Absolute Error:", mae, sep='\n')

Model:	CollabLearner
Root Mean Square Error:
0.06982575649400263
Mean Absolute Error:
0.005146230540069265


# References
https://github.com/microsoft/recommenders

https://github.com/microsoft/recommenders/tree/main/recommenders

https://github.com/microsoft/recommenders/tree/main/examples/00_quick_start

https://github.com/microsoft/recommenders/blob/main/examples/00_quick_start/fastai_movielens.ipynb



X. He et al, Neural Collaborative Filtering, WWW 2017.

Y. Hu et al, Collaborative filtering for implicit feedback datasets, ICDM 2008.

Simple Algorithm for Recommendation (SAR). See notebook sar_deep_dive.ipynb.

Y. Koren and J. Sill, OrdRec: an ordinal model for predicting personalized item rating distributions, RecSys 2011.